In [360]:
%use default

In [388]:
val input = File("input").readLines()

In [389]:
enum class Card() {
    Joker,
    Two,
    Three,
    Four,
    Five,
    Six,
    Seven,
    Eight,
    Nine,
    Ten,
    Queen,
    King,
    Ace;

    companion object {
        operator fun get(face: Char): Card {
            return when (face) {
                'A' -> Card.Ace
                'K' -> Card.King
                'Q' -> Card.Queen
                'J' -> Card.Joker
                'T' -> Card.Ten
                '2' -> Card.Two
                '3' -> Card.Three
                '4' -> Card.Four
                '5' -> Card.Five
                '6' -> Card.Six
                '7' -> Card.Seven
                '8' -> Card.Eight
                '9' -> Card.Nine
                else -> throw Exception("failed to parse card: $face")
            }
        }
    }
}

enum class Kind(count: Int) {
    HighCard(1),
    Pair(2),
    TwoPair(3),
    ThreeOfAKind(4),
    FullHouse(5),
    FourOfAKind(6),
    FiveOfAKind(7);

    companion object {
        operator fun get(cards: List<Card>): Kind {
            var checked = mutableMapOf<Card, Int>()
            for (card in cards) {
                if (checked.contains(card)) continue
                checked[card] = cards.count({ it == card })
            }

            return when (checked.values.maxOrNull()) {
                1 -> Kind.HighCard
                2 -> if (checked.count { it.value == 2 } == 2) Kind.TwoPair else Kind.Pair
                3 -> if (checked.containsValue(2)) Kind.FullHouse else Kind.ThreeOfAKind
                4 -> Kind.FourOfAKind
                5 -> Kind.FiveOfAKind
                else -> throw Exception("couldnt parse card: $cards")
            }
        }
    }
}

In [393]:
data class Hand(var kind: Kind, var cards: List<Card>, val bet: Int) : Comparable<Hand> {
    override fun compareTo(other: Hand): Int {
        if (this.kind != other.kind) {
            val diff = this.kind.ordinal - other.kind.ordinal

            if (diff == 0) throw Exception("encounterd invalid kind comparisons ${this.kind} - ${other.kind}")

            if (diff < 0) {
                return -1
            }

            if (diff > 0) {
                return 1
            }
        }

        for (x in 0..<this.cards.size) {
            if (this.cards[x].ordinal > other.cards[x].ordinal) {
                return 1
            }

            if (this.cards[x].ordinal < other.cards[x].ordinal) {
                return -1
            }

        }

        throw Exception("failed comparisons ${this} - ${other}")
    }

    fun replaceJokers() {
        val jokerCount = this.cards.count { it == Card.Joker }
        if (jokerCount == 0) return

        val filtered = this.cards.filter { it != Card.Joker }
        val counts = filtered
            .distinct()
            .map { it -> filtered.count({ it2 -> it2 == it }) }
            .sortedDescending()
       

        if (jokerCount >= 4) {
            this.kind = Kind.FiveOfAKind
            return
        }
         
        val maxCount = counts[0]

        if (jokerCount == 3) {
            if (maxCount == 2) {
                this.kind = Kind.FiveOfAKind
                return
            }
            this.kind = Kind.FourOfAKind
            return
        }


        if (jokerCount == 2) {
            if (maxCount == 3) {
                this.kind = Kind.FiveOfAKind
                return
            }

            if (maxCount == 2) {
                this.kind = Kind.FourOfAKind
                return
            }

            this.kind = Kind.ThreeOfAKind
            return
        }



        if (maxCount == 4) {
            this.kind = Kind.FiveOfAKind
            return
        }

        if (maxCount == 3) {
            this.kind = Kind.FourOfAKind
            return
        }

        if (maxCount == 2) {
            if (counts[1] == 2 ){
                this.kind = Kind.FullHouse
                return
            }
            
            this.kind = Kind.ThreeOfAKind
            return
        }
        
        this.kind = Kind.Pair 
        return

    }
}

val hands = input.map { it ->
    val split = it.split(" ")
    Hand(
        Kind[(split[0].map { it -> Card[it] })],
        split[0].map { Card[it] },
        split[1].toInt()
    )
}


In [394]:
hands.sorted().foldIndexed(0, { index, acc, it ->
    acc + ((index+1) * it.bet)
})

251532274

In [395]:
hands.forEach({it -> it.replaceJokers()})
hands.sorted().toDataFrame()

<iframe onload="o_resize_iframe_out_19()" style="width:100%;" class="result_container" id="iframe_out_19" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_-486539246"></table>

<p class="dataframe_description">... showing only top 20 of 1000 rows</p><p class="dataframe_description">DataFrame: rowsCount = 1000, columnsCount = 3</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 f

In [396]:
hands.sorted().foldIndexed(0, { index, acc, it ->
    acc + ((index+1) * it.bet)
})

252113488